# A formalism for steering with local quantum measurements
## A. B. Sainz, L. Aolita, M. Piani, M. J. Hoban, P. Skrzypczyk
### https://arxiv.org/abs/1708.00756

This notebook provides details for the numerical results reported in this work

If you wish to run the notebook, rather than just view it, you will need:

- Matlab with the [API for Python](https://www.mathworks.com/help/matlab/matlab-engine-for-python.html) installed.
- The [Calysto Matlab kernel for Python](https://github.com/Calysto/matlab_kernel) 
- [CVX](http://cvxr.com/) Software for Disciplined Convex Programming 
- [QETlab](http://www.qetlab.com/Main_Page) a MATLAB toolbook for quantum entanglement.
- The codes provided in this the github repository (which are generated by running the notebook)

## Preliminary function definitions

### Generate assemblage given state and measurements:

We assume that all measurements are dichotomic in the following, and use the "Collins-Gisin" (CG) representation of an assemblage -- i.e. we use the no-signalling constraints to give a minimal representation of an assemblage. In particular, we only need to give  

\begin{align}
\sigma_R &= \mathrm{tr}_{AB}[\rho^{ABC}] \\
\sigma^A_x &= \mathrm{tr}_{AB}[(M_{0|x} \otimes \mathbb{1} \otimes \mathbb{1})\rho^{ABC}] \quad \forall x\\
\sigma^B_y &= \mathrm{tr}_{AB}[(\mathbb{1} \otimes M_{0|y} \otimes \mathbb{1})\rho^{ABC}] \quad \forall x\\
\sigma_{xy} &= \mathrm{tr}_{AB}[(M_{0|x} \otimes M_{0|y} \otimes \mathbb{I})\rho^{ABC}] \quad \forall x,y
\end{align}


In [ ]:
%%file GenerateAssem.m

function [Sr,Sx,Sy,S] = GenerateAssem(rho, Ma, Mb)
% Given the measurements Ma for Alice, Mb for Bob, and the state rho for
% Alice-Bob-Charlie, generates the assemblage for Charlie in CG form

    [da,~,ma] = size(Ma);
    [db,~,mb] = size(Mb);
    dtot=size(rho,1);
    dc=dtot/(da*db);
    
    Sr = PartialTrace(rho,[1,2],[da,db,dc]);
    
    Sx=zeros(dc,dc,ma);
    for x=1:ma
        Sx(:,:,x)=PartialTrace(Tensor(Ma(:,:,x),eye(db),eye(dc))*rho,[1,2],[da,db,dc]);
    end

    Sy=zeros(dc,dc,mb);
    for y=1:mb
        Sy(:,:,y)=PartialTrace(Tensor(eye(da),Mb(:,:,y),eye(dc))*rho,[1,2],[da,db,dc]);
    end
    
    S=zeros(dc,dc,ma*mb);
    for x=1:ma
        for y=1:mb
            S(:,:,ma*(y-1)+x)=PartialTrace(Tensor(Ma(:,:,x),Mb(:,:,y),eye(dc))*rho,[1,2],[da,db,dc]);
        end
    end
    
end

### Apply the Breur-map to an assemblage

We want to be able to apply the positive but not completely-positive [Breuer map](https://doi.org/10.1103/PhysRevLett.97.080501) $\Lambda[\cdot]$, to an assemblage, where  

$$ \Lambda[\rho] = \tfrac{1}{2}(\mathrm{tr}[\rho]\mathbb{I} - \rho - U\rho^\intercal U^\dagger)$$

acts on states in $\mathbb{C}^4$ and $U$ is an anti-symmetric operator. 

When applied to an assemblage in CG representation, we have 
\begin{align}
\sigma_R' &= \Lambda[\sigma_R]\\
\sigma_x^{A'} &= \Lambda[\sigma_x^A] \quad \forall x \\
\sigma_y^{B'} &= \Lambda[\sigma_y^B] \quad \forall y \\
\sigma_{xy}' &= \Lambda[\sigma_{xy}] \quad \forall x,y
\end{align}

In [ ]:
%%file BreuerMapOnAssem.m

function [Rr, Rx, Ry, R] = BreuerMapOnAssem(U, Sr, Sx, Sy,S,ma,mb)
% Given an antisymmetric unitary U, it applies the map
% Lambda[rho] = 1/2*(trace(rho) Id - rho - U rho' U') 
% to the assemblage to generate a new one. We are assuming a steering 
% scenario with two unstrusted parties, Alice and Bob,  ma/mb = no. 
% of measurements for Alice/Bob and the assemblage is given in CG form

    dc=size(Sr,1);
    Id=eye(dc);
    
    Rr=(trace(Sr)*Id-Sr-U*transpose(Sr)*U')/2;
    
    Rx=zeros(dc,dc,ma);
    for x=1:ma
        Rx(:,:,x)=(trace(Sx(:,:,x))*Id-Sx(:,:,x)-U*transpose(Sx(:,:,x))*U')/2;
    end
   
    Ry=zeros(dc,dc,mb);
    for y=1:mb
        Ry(:,:,y)=(trace(Sy(:,:,y))*Id-Sy(:,:,y)-U*transpose(Sy(:,:,y))*U')/2;
    end
    
    R=zeros(dc,dc,ma*mb);
    for x=1:ma
        for y=1:mb
            R(:,:,ma*(y-1)+x)=(trace(S(:,:,ma*(y-1)+x))*Id-S(:,:,ma*(y-1)+x)-U*transpose(S(:,:,ma*(y-1)+x))*U')/2;
        end
    end

end

### Outer approximation to the set of quantum assemblages

We also need to test membership of an assemblage in the quantum set of assemblages. Since this set has a complicated structure, we will use an outer approximation to it, such that membership can be tested via semidefinite programming. 

Given an assemblage $\sigma_{AB|XY} = \{\sigma_{ab|xy}\}_{a,b,x,y} \equiv \{\sigma_R, \sigma_x^A, \sigma_y^B, \sigma_{xy}\}_{x,y}$ in the case of dichotomic measurements, we solve the following optimisation problem, which is the robustness-based version of the membership problem:

\begin{align}
\min_{p,\pi_{AB|XY}} &\quad p \\
\text{s.t.}&\quad \frac{\sigma_{AB|XY} + p \pi_{AB|XY}}{1+p} \in \tilde{\mathcal{Q}} \\
&\quad p \geq 0
\end{align}
where $\tilde{\mathcal{Q}}$ is an approximation to the quantum set of assemblages. If the solution to this problem is $p = 0$, then $\sigma_{AB|XY}$ belongs to the quantum set of assemblages. If $p > 0$, then $\sigma_{AB|XY} \notin \tilde{\mathcal{Q}}$, as it is necessary to add a non-zero amount of noise $\pi_{AB|XY}$ in order to bring the assemblage inside the quantum set. 

In this work we use the [Almost-Quantum](https://doi.org/10.1103/PhysRevLett.115.190403) (AQ) SDP relaxation of the quantum set. This test asks that there exists a positive-semidefinite moment matrix $\Gamma \geq 0$, which satisfies constraints which arise from the algebra of projectors, and which is consistent with the observed assemblage. The explicit details of these constraints can be found [here](https://doi.org/10.1103/PhysRevLett.115.190403). 

Because of the duality theory of semidefinite programs, if the assemblage is outside the AQ set of assemblages, there exists a linear functional that certies this fact, i.e. a set of operators $\{F_R, F_x^A, F_y^B, F_{xy}\}_{x,y}$, such that  

$$ \beta = \mathrm{tr}\left[\sigma_R F_R + \sum_x \sigma_x^A F_x^A + \sum_y \sigma_y^B F_y^B + \sum_{xy} \sigma_{xy}F_{xy} \right] > 0$$
while for all quantum assemblages $\beta \leq 0$. 

In [ ]:
%%file IsAQAssemblage.m

function [output,Fr,Fx,Fy,F,beta,Gamma] = IsAQAssemblage(Sr,Sx,Sy,S,ma,mb)

% given an assemblage in CG notation, this function determines whether it belongs to the 
% Almost-Quantum outer approximation to the set of quantum assemblages. 
% This function returns out=0 if the assemblage is outside and out=1 if inside.
%
% Steering scenario with two uncharacterised parties (Alice and Bob), and
% one characterised one (Charlie).
%
% mb = number of measurements for Alice
% mc = number of measurements for Bob
%
% this code assumes that all measurements are dichotomic


d=size(Sr,1); 
dimG=(ma+1)*(mb+1)*d; 

cvx_begin sdp quiet

    variable Gamma(dimG,dimG) hermitian % moment matrix
    
    variable pi_r(d,d) hermitian 
    variable pix(d,d,ma) hermitian 
    variable piy(d,d,mb) hermitian 
    variable pixy(d,d,ma*mb) hermitian semidefinite

    dual variable Fr
    dual variables Fx{ma}
    dual variables Fy{mb}
    dual variables F{ma*mb}
            
    minimise trace(pi_r)
            
    Gamma == hermitian_semidefinite(dimG);
                
    for x = 1:ma
        for y = 1:mb
            pix(:,:,x)-pixy(:,:,x+ma*(y-1)) == hermitian_semidefinite(d);
            piy(:,:,y)-pixy(:,:,x+ma*(y-1)) == hermitian_semidefinite(d);
            pi_r-pix(:,:,x)-piy(:,:,y)+pixy(:,:,x+ma*(y-1)) == hermitian_semidefinite(d);
        end
    end
            
    % First row
    Fr : Gamma(1:d,1:d)==Sr+pi_r;
                
    % Alice's marginal assemblage
    for k=1:ma
        Fx{k} : Gamma(1:d,k*d+1:(k+1)*d)==Sx(:,:,k)+pix(:,:,k);
    end
                
    % Bob's marginal assemblage
    for k=1:mb
        Fy{k} : Gamma(1:d,k*d*(ma+1)+1:k*d*(ma+1)+d)==Sy(:,:,k) + piy(:,:,k);
    end

    % the assemblage for all-but-one outcome
    for k=1:ma
        for j=1:mb
            F{ma*(j-1)+k} : Gamma(1:d,j*d*(ma+1)+k*d+1:j*d*(ma+1)+k*d+d)...
                == S(:,:,ma*(j-1)+k) + pixy(:,:,ma*(j-1)+k);
        end
    end
            
    % Conditions on Gamma to be AQ                
    for y=0:ma
        for z=0:mb
            for yp=0:ma
                for zp=0:mb 
                    if y==0 && z==0 % reductions of type AB, where it can be A=1 and/or B=1
                        r1=1;
                        c1=zp*d*(ma+1)+yp*d+1;
                        r2=yp*d+1;
                        c2=zp*d*(ma+1)+1;
                        r3=yp*d+1;
                        c3=zp*d*(ma+1)+yp*d+1;
                        r4=zp*d*(ma+1)+1;
                        c4=zp*d*(ma+1)+yp*d+1;
                        r5=zp*d*(ma+1)+yp*d+1;
                        c5=zp*d*(ma+1)+1;
                        r6=zp*d*(ma+1)+yp*d+1;
                        c6=zp*d*(ma+1)+yp*d+1;
                        r7=zp*d*(ma+1)+1;
                        c7=yp*d+1;
                        r8=zp*d*(ma+1)+yp*d+1;
                        c8=1;
                        r9=zp*d*(ma+1)+yp*d+1;
                        c9=yp*d+1;
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r2:r2+d-1,c2:c2+d-1);
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r3:r3+d-1,c3:c3+d-1);
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r4:r4+d-1,c4:c4+d-1);
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r5:r5+d-1,c5:c5+d-1);
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r6:r6+d-1,c6:c6+d-1);
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r7:r7+d-1,c7:c7+d-1);
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r8:r8+d-1,c8:c8+d-1);
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r9:r9+d-1,c9:c9+d-1);
                    end
                    
                    if y==0 && z~=0 && yp~=0 && zp~=0 % Reduction of type ABB', none = 1. 
                        r1=z*d*(ma+1)+1;
                        c1=zp*d*(ma+1)+yp*d+1;
                        r2=z*d*(ma+1)+yp*d+1;
                        c2=zp*d*(ma+1)+1;
                        r3=z*d*(ma+1)+yp*d+1;
                        c3=zp*d*(ma+1)+yp*d+1;
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r2:r2+d-1,c2:c2+d-1);
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r3:r3+d-1,c3:c3+d-1);
                    end
                    
                    if z~=0 && yp==0 && zp~=0 % (AB,A')=(A'B,A)', B can be =1.
                        if z~=zp
                            r1=z*d*(ma+1)+y*d+1;
                            c1=zp*d*(ma+1)+1;
                            r2=zp*d*(ma+1)+y*d+1;
                            c2=z*d*(ma+1)+1;
                            Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r2:r2+d-1,c2:c2+d-1)';
                        end
                    end
                    
                    if z==0 && y~=0 && yp~=0 && zp~=0 % Reduction of type AA'B, none = 1.
                        r1=y*d+1;
                        c1=zp*d*(ma+1)+yp*d+1;
                        r2=zp*d*(ma+1)+y*d+1;
                        c2=yp*d+1;
                        r3=zp*d*(ma+1)+y*d+1;
                        c3=zp*d*(ma+1)+yp*d+1;
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r2:r2+d-1,c2:c2+d-1);
                        Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r3:r3+d-1,c3:c3+d-1);
                    end
                    
                    if y~=0 && yp~=0 && zp==0 % (AB,B')=(AB',B)'
                        if y~=yp
                            r1=z*d*(ma+1)+y*d+1;
                            c1=yp*d+1;
                            r2=z*d*(ma+1)+yp*d+1;
                            c2=y*d+1;
                            Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r2:r2+d-1,c2:c2+d-1)';
                        end                        
                    end
                    
                    if y~=0 && z~=0 && yp~=0 && zp==0 % (AB, A'B') = (A'B',AB)'
                        if y~=yp && z~=zp
                            r1=z*d*(ma+1)+y*d+1;
                            c1=zp*d*(ma+1)+yp*d+1;
                            r2=zp*d*(ma+1)+yp*d+1;
                            c2=z*d*(ma+1)+y*d+1;
                            Gamma(r1:r1+d-1,c1:c1+d-1)==Gamma(r2:r2+d-1,c2:c2+d-1)';
                        end                                            
                    end
                end
            end
        end
    end
    
    cvx_end
    
    threshold = 1e-8; % threshold for being 'inside' AQ (i.e. anything smaller will be considered inside, anything larger outside)
    
    output=cvx_optval <= threshold;
    beta = cvx_optval;
    
end

## Example 1: Breuer Map

The first example will be to use the state in $\mathbb{C}^2 \otimes \mathbb{C}^2 \otimes \mathbb{C}^4$ given in (38). For measurements we will use the CHSH measurements, and for the positive map we will use the Breuer map. We will see that this leads to post-quantum steering

In [96]:
% Initialise state: 

Id16 = eye(16);

psi=Id16(:,2)+Id16(:,3)+Id16(:,5)+Id16(:,9);
psi=psi+1i*(Id16(:,4)+Id16(:,6)+Id16(:,7)+Id16(:,10)+Id16(:,11)+Id16(:,13));
psi=psi-(Id16(:,8)+Id16(:,12)+Id16(:,14)+Id16(:,15));
psi=psi/norm(psi);
rho=psi*psi';

% measurements

ma = 2;
mb = 2;

Ma = zeros(2,2,ma);
Mb = zeros(2,2,mb);

Ma(:,:,1) = (Pauli(4)+Pauli(1))/2;
Ma(:,:,2) = (Pauli(4)+Pauli(3))/2;

Mb(:,:,1) = (Pauli(4)+(Pauli(1)+Pauli(3))/sqrt(2))/2;
Mb(:,:,2) = (Pauli(4)+(-Pauli(1)+Pauli(3))/sqrt(2))/2;

% generate assemblage

[Sr,Sx,Sy,S] = GenerateAssem(rho, Ma, Mb);

% check for membership in AQ before applying map

output = IsAQAssemblage(Sr,Sx,Sy,S,ma,mb)

% check for membership in AQ after applying Breuer map

U = Tensor(Pauli(1),Pauli(2));
[Rr, Rx, Ry, R] = BreuerMapOnAssem(U, Sr, Sx, Sy,S,ma,mb);

[output,Fr,Fx,Fy,F,beta] = IsAQAssemblage(Rr,Rx,Ry,R,ma,mb);
output
beta

Fr
celldisp(Fx)
celldisp(Fy)
celldisp(F)

output =
  logical
   1
output =
  logical
   0
beta =
    0.0210
Fr =
  -0.3684 + 0.0000i  -0.0631 + 0.2282i   0.0753 + 0.3295i   0.0000 - 0.0000i
  -0.0631 - 0.2282i  -0.5822 + 0.0000i   0.0000 + 0.0000i   0.0753 + 0.3295i
   0.0753 - 0.3295i   0.0000 - 0.0000i  -0.5822 + 0.0000i   0.0631 - 0.2282i
   0.0000 + 0.0000i   0.0753 - 0.3295i   0.0631 + 0.2282i  -0.3684 + 0.0000i
Fx{1} =
  -0.0000 + 0.0056i  -0.0571 + 0.0336i   0.0179 + 0.2146i   0.0000 + 0.1057i
   0.3096 + 0.0336i   0.0000 - 0.0168i   0.0000 - 0.1195i  -0.1686 + 0.2146i
  -0.3193 + 0.2146i  -0.0000 - 0.1195i   0.0000 + 0.0168i  -0.1834 - 0.0336i
  -0.0000 + 0.1057i  -0.1328 + 0.2146i  -0.0691 - 0.0336i  -0.0000 - 0.0056i
Fx{2} =
  -0.0508 - 0.0000i  -0.0001 + 0.0223i  -0.0002 + 0.1814i   0.0210 - 0.0001i
   0.0001 + 0.3979i   0.0270 + 0.0000i   0.0318 - 0.0001i   0.0001 + 0.2114i
   0.0002 - 0.0590i  -0.0318 - 0.0001i   0.0270 - 0.0000i   0.0001 + 0.0343i
  -0.0210 - 0.0001i  -0.0001 - 0.0290i  -0.0001 - 0.3413i  -0.0508

### Example 2: Choi map

Our second example will be to use as choice of positive map the [Choi map][1] on $\mathbb{C}^3$. 

As a state, we will consider a pure state in $\mathbb{C}^2 \otimes \mathbb{C}^2 \otimes \mathbb{C}^3$ which is the Dicke state with 2 excitations

$$ \left|\psi\right\rangle = \frac{1}{2}\left(\left|011\right\rangle + \left|002\right\rangle + \left|101\right\rangle + \left|110\right\rangle\right)$$

We will use the same (CHSH) measurements as in the previous example. 

[1]: https://doi.org/10.1016/0024-3795(75)90075-0

In [97]:
clear

% initialise a random state

% we assume the state is 2 x 2 x 3 with Charlie the trusted party

psi = zeros(12,1); % |psi> = (|002> + |011> + |101> + |110>)/2
psi(3) = 1;        % (2 excitation Dicke state)
psi(5) = 1;
psi(8) = 1;
psi(10) = 1;
rho = psi*psi';
rho = rho/trace(rho);

% we can prepare the opeator O by applying the Choi map to C

rhoC = PartialMap(rho,ChoiMap,2,[4,3]);

% initialise randomly chosen projective measurements

ma = 2;
mb = 2;

Ma = zeros(2,2,ma);
Mb = zeros(2,2,mb);

Ma(:,:,1) = (Pauli(4)+Pauli(1))/2;
Ma(:,:,2) = (Pauli(4)+Pauli(3))/2;

Mb(:,:,1) = (Pauli(4)+(Pauli(1)+Pauli(3))/sqrt(2))/2;
Mb(:,:,2) = (Pauli(4)+(-Pauli(1)+Pauli(3))/sqrt(2))/2;

% generate assemblages

[Sr,Sx,Sy,S] = GenerateAssem(rho,Ma,Mb);
[Rr,Rx,Ry,R] = GenerateAssem(rhoC,Ma,Mb);

% check for membership in AQ before applying map

output = IsAQAssemblage(Sr,Sx,Sy,S,ma,mb)

% check for membership in AQ after applying Breuer map

[output, Fr, Fx, Fy, F, beta,Gam] = IsAQAssemblage(Rr,Rx,Ry,R,ma,mb);
output
beta

Fr
celldisp(Fx)
celldisp(Fy)
celldisp(F)

output =
  logical
   1
output =
  logical
   0
beta =
    0.0110
Fr =
   -0.0762    0.1437    0.0126
    0.1437   -0.2713   -0.0240
    0.0126   -0.0240   -0.0121
Fx{1} =
   -0.0000   -0.0255    0.0232
   -0.5495   -0.0000    0.0438
   -0.0232    0.0523    0.0000
Fx{2} =
   -0.0007   -0.0000    0.0243
    0.0000    0.0000    0.0000
    0.0281    0.0000   -0.9850
Fy{1} =
   -0.1152   -0.1026    0.2165
   -0.0362    0.1280    0.0019
    0.2089   -0.0276   -0.2757
Fy{2} =
    0.0329   -0.0206   -0.0476
   -0.1314    0.1433   -0.0021
   -0.0775    0.0757   -0.2397
F{1} =
    0.1481         0         0
    0.2907    0.0152         0
   -0.5505   -0.0479    0.0360
F{2} =
    0.0051         0         0
    0.0142   -0.0000         0
   -0.1918   -0.4704    0.7562
F{3} =
   -0.1481         0         0
    0.2907   -0.0152         0
    0.5505   -0.0479   -0.0360
F{4} =
    0.0051         0         0
   -0.0142   -0.0000         0
   -0.1918    0.4704    0.7562
